Leung Wai Liu <br>
JPMC-SMM4H <br>
July 14, 2022 <br>
Task 5 Weighted Average Ensembling

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from labels_to_ids import task7_labels_to_ids
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score, confusion_matrix
import os

In [2]:
# Loading up all the predictions data

n_rnds = 5
original_df = pd.read_csv('../Datasets/dev.tsv', sep='\t')

models = ['dccuchile/bert-base-spanish-wwm-uncased', 'dccuchile/bert-base-spanish-wwm-cased', 'xlm-roberta-base', 'bert-base-multilingual-uncased', 'bert-base-multilingual-cased']
n_models = len(models)

epoch_string = '../15_epochs_small_model/eval_testing/saved_eval_test_result_5'
n_rows = len(original_df)

labels_to_ids = task7_labels_to_ids
ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

# Loading up all of the results
best_f1 = pd.read_csv('../15_epochs_small_model/training_predictions/validating_statistics/all_best_f1_score.tsv', sep='\t')
best_f1



,Unnamed: 0,dccuchile/bert-base-spanish-wwm-uncased,dccuchile/bert-base-spanish-wwm-cased,xlm-roberta-base,bert-base-multilingual-uncased,bert-base-multilingual-cased
0,0,0.706554,0.722016,0.712117,0.732637,0.716880
1,1,0.714637,0.698767,0.718595,0.720507,0.718697
2,2,0.706063,0.697280,0.711941,0.725631,0.717069
3,3,0.705547,0.696304,0.710646,0.721070,0.715650
4,4,0.687831,0.710475,0.716102,0.719317,0.712978


In [7]:
# Retrieving all the predictions from the 
list_of_df = []
list_of_f1_score = []
sum_of_all_f1_score = 0

for model in models: 
    specific_model_row = []
    specific_model_row_f1 = []
    for rnd in range(n_rnds):
        to_read_string = epoch_string + '/' + model + '/' + str(rnd) + '/unformatted_eval_test_result.tsv'
        
        specific_f1_score = best_f1.at[rnd, model]
        specific_model_row_f1.append(specific_f1_score)
        sum_of_all_f1_score += specific_f1_score
        
        particular_model_df = pd.read_csv(to_read_string, sep='\t')
        specific_model_row.append(particular_model_df)
    
    list_of_df.append(specific_model_row)
    list_of_f1_score.append(specific_model_row_f1)


17.81530721144871


In [9]:
# TAKING THE WEIGHTED AVERAGE OF DATA

weighted_avg_original_tweet_id_list = []
weighted_avg_original_sentence_list = []
weighted_avg_original_label_list = []
weighted_avg_original_numbered_label_list = []

weighted_avg_predicted_number_results = []
weighted_avg_predicted_results = []


for index, row in original_df.iterrows(): 
    # getting the original values in the tweet
    original_tweet_id = row['tweet_id']
    original_sentence = row['tweet_text']
    original_label = row['label']
    
    # transferring the labels over to final list
    weighted_avg_original_tweet_id_list.append(original_tweet_id)
    weighted_avg_original_sentence_list.append(original_sentence)
    weighted_avg_original_label_list.append(original_label)
    
    specific_row_value = 0.0
    # go through every models' row of data 
    
    for model_num in range(n_models):
        for rnd_num in range(n_rnds):

            particular_row_df = list_of_df[model_num][rnd_num]
            row = particular_row_df.loc[(particular_row_df['tweet_id'] == original_tweet_id) & (particular_row_df['text'] == original_sentence)]

            prediction = labels_to_ids[row['label'].values[0]] * list_of_f1_score[model_num][rnd_num]
            specific_row_value += prediction

    specific_row_value = specific_row_value / sum_of_all_f1_score
    specific_row_result = int(round(specific_row_value))
    weighted_avg_predicted_results.append(ids_to_labels[specific_row_result])
    

2.0
2
2.0
2
2.0
2
2.0
2
2.0
2
1.0
1
1.8800581588603922
2
2.0
2
2.0
2
2.0
2
0.3628900309387988
0
2.0
2
2.0
2
2.0
2
2.0
2
0.0
0
2.0
2
0.0
0
1.0
1
2.0
2
2.0
2
1.921721248047112
2
2.0
2
2.0
2
1.9191488842191753
2
0.0
0
2.0
2
0.6434367462708296
1
0.5589881972482246
1
0.20269164374178378
0
2.0
2
1.9597498273176943
2
0.0
0
1.8406365485025957
2
1.9603676475942888
2
1.0
1
2.0
2
2.0
2
2.0
2
2.0
2
2.0
2
0.0
0
1.639650219477086
2
0.7202368726611438
1
2.0
2
0.23710035238046767
0
2.0
2
0.04019585488937316
0
2.0
2
2.0
2
2.0
2
2.0
2
2.0
2
2.0
2
2.0
2
2.0
2
0.683036938917566
1
2.0
2
0.2779295819927469
0
2.0
2
2.0
2
2.0
2
2.0
2
2.0
2
1.281569173541126
1
2.0
2
2.0
2
0.40165396885618987
0
2.0
2
2.0
2
2.0
2
1.0399721728447984
1
2.0
2
2.0
2
1.9608606240235562
2
2.0
2
1.0000984125181138
1
2.0
2
0.0
0
0.0
0
0.0
0
0.2406481687621795
0
2.0
2
2.0
2
0.12004980074948707
0
2.0
2
1.3226552929852724
1
1.1620458189182694
1
0.03913937597644399
0
0.0
0
0.07959917982556705
0
2.0
2
2.0
2
0.0
0
2.0
2
2.0
2
2.0
2
0.0
0
0.0


KeyboardInterrupt: 

In [5]:
# Calculating sklearn metrics

weighted_avg_original_numbered_label_list = [labels_to_ids[label] for label in weighted_avg_original_label_list]
weighted_avg_predicted_number_results = [labels_to_ids[label] for label in weighted_avg_predicted_results]

accuracy_result = accuracy_score(weighted_avg_original_numbered_label_list, weighted_avg_predicted_number_results)
f1_result = f1_score(weighted_avg_original_numbered_label_list, weighted_avg_predicted_number_results, labels=[0], average=None)[0]
precision_result = precision_score(weighted_avg_original_numbered_label_list, weighted_avg_predicted_number_results, labels=[0], average=None)[0]
recall_result = recall_score(weighted_avg_original_numbered_label_list, weighted_avg_predicted_number_results, labels=[0], average=None)[0]

confusion_matrix_result = confusion_matrix(weighted_avg_original_numbered_label_list, weighted_avg_predicted_number_results)
classification_result = classification_report(weighted_avg_original_numbered_label_list, weighted_avg_predicted_number_results, output_dict=True)

print("ACCURACY:", accuracy_result)
print("F1:", f1_result)
print("PRECISION:", precision_result)
print("RECALL", recall_result)

# Saving results to file
os.makedirs('../15_epochs_small_model/eval_testing/eval_validation_statistics/weighted_avg_ensemble', exist_ok=True)

with open('../15_epochs_small_model/eval_testing/eval_validation_statistics/weighted_avg_ensemble/weighted_avg_ensemble_valid_stats.txt', 'w') as file:
        file.write("Accuracy: " + str(accuracy_result) + "\n")
        file.write("F1 Score (of self_reports): " + str(f1_result) + "\n")
        file.write("Precison Score (of self_reports): " + str(precision_result) + "\n")
        file.write("Recall result (of self_reports): " + str(recall_result) + "\n")


cm_df = pd.DataFrame(confusion_matrix_result)
cr_df = pd.DataFrame(classification_result).transpose()

cm_df.to_csv('../15_epochs_small_model/eval_testing/eval_validation_statistics/weighted_avg_ensemble/weighted_avg_confusion_matrix.tsv', sep='\t')
cr_df.to_csv('../15_epochs_small_model/eval_testing/eval_validation_statistics/weighted_avg_ensemble/weighted_avg_classification_report.tsv', sep='\t')




ACCURACY: 0.8405970149253731
F1: 0.7452229299363057
PRECISION: 0.6603951081843838
RECALL 0.8550548112058465


In [6]:
# Saving it as a dataframe
unformatted_weighted_avg_prediction_data = pd.DataFrame(zip(weighted_avg_original_tweet_id_list, weighted_avg_original_sentence_list, weighted_avg_original_label_list, weighted_avg_predicted_results), columns=['tweet_id', 'text', 'Orig', 'label'])
formatted_weighted_avg_prediction_data = unformatted_weighted_avg_prediction_data.drop(columns = ['Orig'])

# Saving it as a tsv file
os.makedirs('../15_epochs_small_model/eval_testing/eval_validation_statistics/', exist_ok=True)
unformatted_weighted_avg_prediction_data.to_csv('../15_epochs_small_model/eval_testing/eval_validation_statistics/weighted_avg_ensemble/unformatted_weighted_avg_data.tsv', sep='\t', index=False)
formatted_weighted_avg_prediction_data.to_csv('../15_epochs_small_model/eval_testing/eval_validation_statistics/weighted_avg_ensemble/formatted_weighted_avg_data.tsv', sep='\t', index=False)